In [7]:
def warn(*args, **kwargs):
    pass

import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

In [ ]:
# OLLAMA_HOST=127.0.0.1:11500 ollama pull llama3
# ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIGIBF4ybPVTD9MroHmY6kKkkWP0sLdW3JfmY2cQAkX9S
# nous-hermes-2-mistral-7b-dpo

In [ ]:
# Import Libraries
import os
# os.environ["OLLAMA_HOST"] = "127.0.0.1:11500"
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_community.llms import ollama
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, AIMessage
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings

import pickle

In [ ]:
# Load and Process Documents

directory = "../docs/"
all_docs = []

for filename in os.listdir(directory):
    if filename.endswith(".pdf"):
        file_path = os.path.join(directory, filename)
        loader = PyPDFLoader(file_path)
        documents = loader.load()
        all_docs.extend(documents)


In [10]:
# Chunking

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunks = text_splitter.split_documents(all_docs)
